In [10]:
# STEP 1: Unzip Nested ZIPs
import zipfile, os

main_zip = 'ecommercefunnel.zip'
extract_folder = 'unzipped_csvs'
os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(main_zip, 'r') as outer_zip:
    for name in outer_zip.namelist():
        if name.endswith('.zip'):
            with outer_zip.open(name) as inner_zip_file:
                with zipfile.ZipFile(inner_zip_file) as inner_zip:
                    for csv_name in inner_zip.namelist():
                        if csv_name.endswith('.csv'):
                            inner_zip.extract(csv_name, path=extract_folder)


In [5]:
!pip install pyspark

In [11]:
# STEP 2: PySpark Session + Load CSVs
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, collect_set, array_contains

spark = SparkSession.builder.appName("FunnelAnalysis").getOrCreate()
df = spark.read.option("header", True).csv(f"{extract_folder}/*.csv")
df = df.withColumn("event_time", to_timestamp("event_time"))


In [12]:
# STEP 3: Clean and Filter for Funnel
df = df.dropna(subset=["user_id", "user_session"])
funnel_df = df.filter(col("event_type").isin(["view", "cart", "purchase"]))


In [13]:
# STEP 4: Build Session-Level Funnel
session_events = funnel_df.groupBy("user_session").agg(collect_set("event_type").alias("events"))

view_sessions = session_events.filter(array_contains(col("events"), "view")).count()
cart_sessions = session_events.filter(array_contains(col("events"), "cart")).count()
purchase_sessions = session_events.filter(array_contains(col("events"), "purchase")).count()


In [14]:
# STEP 5: Visualize Funnel in Plotly
import plotly.graph_objects as go

stages = ['View', 'Cart', 'Purchase']
counts = [view_sessions, cart_sessions, purchase_sessions]

fig = go.Figure(go.Funnel(
    y=stages,
    x=counts,
    textinfo="value+percent initial"
))

fig.update_layout(title="E-commerce Funnel Drop-off Analysis")
fig.show()

print(f"View → Cart: {100 * (1 - cart_sessions/view_sessions):.2f}% dropped")
print(f"Cart → Purchase: {100 * (1 - purchase_sessions/cart_sessions):.2f}% dropped")


View → Cart: 76.97% dropped
Cart → Purchase: 84.21% dropped


## Reduce Cart-to-Purchase Drop-Off (Only ~4% of Cart → Purchase)

- **Streamline Checkout Flow**  
  - Remove unnecessary form fields  
  - Offer a “guest checkout” option  
  - Pre-fill user data where possible  

- **Build In-Cart Incentives**  
  - Show limited-time promos (e.g. 5% off) when items are added  
  - Highlight low-stock warnings to create urgency  

- **Remarketing & Abandoned-Cart Emails**  
  - Send reminder emails/SMS within 24 h of abandonment  
  - Include personalized product images and one-click “return to cart” links

In [17]:
from pyspark.sql.functions import (
    date_format, countDistinct, count, when, lit,
    min as spark_min, max as spark_max, unix_timestamp
)
import plotly.express as px

# Monthly Funnel Trends
monthly_funnel = funnel_df \
    .withColumn("event_month", date_format("event_time", "yyyy-MM")) \
    .groupBy("event_month", "event_type") \
    .agg(countDistinct("user_session").alias("sessions")) \
    .orderBy("event_month", "event_type")

monthly_pd = monthly_funnel.toPandas()
fig_monthly = px.line(
    monthly_pd,
    x="event_month", y="sessions", color="event_type",
    title="Monthly Funnel Progression",
    labels={"sessions": "Unique Sessions", "event_month": "Month", "event_type": "Stage"}
)
fig_monthly.show()


## Leverage Temporal Trends & Seasonality

- **Holiday-Safe UI**  
  - December dip in conversions → optimize page load times, offer gift bundles, clear holiday messaging  

- **Post-Holiday Re-engagement**  
  - January bounce in views but lagging purchases → run “New Year, New You” campaigns, time-sensitive discounts  


In [19]:
# Conversion Rate by Category
category_funnel = funnel_df \
    .filter(col("event_type").isin("view", "purchase")) \
    .groupBy("category_code", "event_type") \
    .agg(countDistinct("user_session").alias("sessions"))

category_pivot = category_funnel.groupBy("category_code") \
    .pivot("event_type", ["view", "purchase"]) \
    .sum("sessions").fillna(0) \
    .withColumn(
        "conversion_rate",
        when(col("view") > 0, col("purchase") / col("view")).otherwise(lit(0))
    )

cat_pd = category_pivot.toPandas().sort_values("conversion_rate", ascending=False)
fig_cat = px.bar(
    cat_pd.head(5),
    x="category_code", y="conversion_rate",
    title="Top 5 Categories by Conversion Rate",
    labels={"conversion_rate": "Conversion Rate", "category_code": "Category"}
)
fig_cat.update_layout(xaxis_tickangle=-45)
fig_cat.show()

## Improve Category & Brand Conversion

- **Double Down on High-Performing Categories**  
  - E.g. `apparel_glove` (~33% conversion) → increase ad spend and promotions  
  - Bundle slower-moving SKUs with top categories  

- **Optimize Low-Conversion Categories**  
  - E.g. `accessories_cosmetic_bag` (~6% conversion) → A/B test page layouts, add richer media or reviews  


In [20]:
# Top Brands by Conversion Rate
brand_funnel = funnel_df \
    .filter(col("event_type").isin("view", "purchase")) \
    .groupBy("brand", "event_type") \
    .agg(countDistinct("user_session").alias("sessions"))

brand_pivot = brand_funnel.groupBy("brand") \
    .pivot("event_type", ["view", "purchase"]) \
    .sum("sessions").fillna(0) \
    .withColumn(
        "conversion_rate",
        when(col("view") > 0, col("purchase") / col("view")).otherwise(lit(0))
    )

brand_pd = brand_pivot.toPandas().sort_values("conversion_rate", ascending=False)
fig_brand = px.bar(
    brand_pd.head(20),
    x="brand", y="conversion_rate",
    title="Top 20 Brands by Conversion Rate",
    labels={"conversion_rate": "Conversion Rate", "brand": "Brand"}
)
fig_brand.update_layout(xaxis_tickangle=-45)
fig_brand.show()


- **Partner with Top Brands**  
  - Brands like **eunyul**, **supertan**, **elskin** (40–44% conversion) → negotiate featured placements, exclusive bundles  

- **Re-evaluate Underperforming Brands**  
  - Refresh content/pages or run targeted promotions for brands below ~20% conversion  

In [21]:
# Session Duration Distribution (View → Purchase)
view_times = funnel_df \
    .filter(col("event_type") == "view") \
    .groupBy("user_session") \
    .agg(spark_min("event_time").alias("view_time"))

purchase_times = funnel_df \
    .filter(col("event_type") == "purchase") \
    .groupBy("user_session") \
    .agg(spark_max("event_time").alias("purchase_time"))

session_time = view_times.join(purchase_times, "user_session") \
    .withColumn(
        "duration_minutes",
        (unix_timestamp("purchase_time") - unix_timestamp("view_time")) / 60
    )

dur_pd = session_time.select("duration_minutes").toPandas()
fig_dur = px.histogram(
    dur_pd, x="duration_minutes",
    nbins=50,
    title="Distribution of Session Conversion Times",
    labels={"duration_minutes": "Duration (minutes)"}
)
fig_dur.show()

## Streamline Session Conversion Time

- **Speed Up Conversion**  
  - Most purchases occur within minutes; outliers stretch to hours/days  
  - Implement “Save for Later” and persistent carts  

- **Timed Reminders**  
  - Send push notifications or emails within 1–2 h of first view  

In [22]:
# Monthly Cart Abandonments
cart_sessions_month = funnel_df \
    .filter(col("event_type") == "cart") \
    .select("user_session", date_format("event_time", "yyyy-MM").alias("event_month")) \
    .distinct()

purchase_sessions_only = funnel_df \
    .filter(col("event_type") == "purchase") \
    .select("user_session").distinct()

monthly_abandon = cart_sessions_month.join(
    purchase_sessions_only, on="user_session", how="left_anti"
).groupBy("event_month") \
 .agg(count("*").alias("abandoned_carts")) \
 .orderBy("event_month")

abandon_pd = monthly_abandon.toPandas()
fig_abandon = px.bar(
    abandon_pd,
    x="event_month", y="abandoned_carts",
    title="Monthly Cart Abandonments",
    labels={"abandoned_carts": "Abandoned Carts", "event_month": "Month"}
)
fig_abandon.show()

## Tackle Cart Abandonments by Month

- **October 2019 Peak** (~205 k abandons)  
  - Audit UI or pricing changes rolled out in October  
  - Compare user paths in October vs. other months to uncover friction  

- **Monitor Seasonal Shipping & Returns**  
  - Align cart-abandonment spikes with shipping cut-offs or return-policy queries  
  - Communicate clear shipping deadlines and free returns  

In [24]:
from pyspark.sql.functions import (
    countDistinct, avg, col, when, lit
)
from pyspark.sql.types import FloatType

# Session Duration Summary (25th, 50th, 75th percentiles)
quantiles = session_time.approxQuantile("duration_minutes", [0.25, 0.5, 0.75], 0)
print("Session Duration (minutes) — 25th, 50th, 75th percentiles:", quantiles)

Session Duration (minutes) — 25th, 50th, 75th percentiles: [6.883333333333334, 16.716666666666665, 40.35]


## Optimize Timing of Reminders  
- **Insight**: 50% of converting sessions complete within ~17 minutes; 75% within ~40 minutes.  
- **Action**:  
  - Trigger **in-session nudges** (e.g. “Still interested?” banners) after ~7 minutes and ~17 minutes.  
  - Send a **push notification or email** around the 15–20 minute mark to users who added items to cart but haven’t checked out.

---

In [28]:
# Category‐Level Funnel Metrics
cat_stats = (
    funnel_df
    .filter(col("category_code").isNotNull() & (col("category_code") != "NULL"))
    .groupBy("category_code")
    .pivot("event_type", ["view", "cart", "purchase"])
    .agg(countDistinct("user_session"))
    .fillna(0)
    .withColumn("view_to_cart", when(col("view") > 0, col("cart") / col("view")).otherwise(lit(0)))
    .withColumn("cart_to_purchase", when(col("cart") > 0, col("purchase") / col("cart")).otherwise(lit(0)))
    .withColumn("view_to_purchase", when(col("view") > 0, col("purchase") / col("view")).otherwise(lit(0)))
)

# top 10 categories by view→cart conversion
cat_stats.orderBy(col("view_to_cart").desc()).show(10, truncate=False)

+--------------------------------------+-----+-----+--------+-------------------+-------------------+--------------------+
|category_code                         |view |cart |purchase|view_to_cart       |cart_to_purchase   |view_to_purchase    |
+--------------------------------------+-----+-----+--------+-------------------+-------------------+--------------------+
|apparel.glove                         |13574|10478|4433    |0.7719168999557978 |0.4230769230769231 |0.326580226904376   |
|stationery.cartrige                   |14316|6950 |2839    |0.48547080189997205|0.4084892086330935 |0.1983095836825929  |
|appliances.environment.air_conditioner|386  |117  |35      |0.30310880829015546|0.29914529914529914|0.09067357512953368 |
|furniture.bathroom.bath               |10727|3185 |988     |0.2969143283303813 |0.31020408163265306|0.09210403654330195 |
|accessories.cosmetic_bag              |1878 |429  |112     |0.22843450479233227|0.26107226107226106|0.059637912673056445|
|appliances.envi

## Double Down on High-Performing Categories  
- **Insight**:  
  - `apparel.glove` → 77% view→cart, 42% cart→purchase, overall ~33% conversion  
  - `stationery.cartrige` → 49% view→cart, 41% cart→purchase (~20% overall)  
- **Action**:  
  - Increase ad spend and homepage real estate for these categories.  
  - Offer “bundle” deals that combine gloves or cartridges with related accessories to boost average order value.


In [26]:
# Correlation Between Price and Conversion
# Cast price to float
df = df.withColumn("price", col("price").cast(FloatType()))

# Compute avg purchase price per category
cat_price = df.filter(col("event_type")=="purchase") \
    .groupBy("category_code") \
    .agg(avg("price").alias("avg_price"))

# Join with conversion stats and compute correlation in pandas
cat_corr = cat_stats.join(cat_price, "category_code", "left")
cat_corr_pd = cat_corr.toPandas()
print("Correlation avg_price vs view_to_cart:",
      cat_corr_pd["avg_price"].corr(cat_corr_pd["view_to_cart"]))

Correlation avg_price vs view_to_cart: -0.55822136691285


### Counteract Price Sensitivity  
- **Insight**: Average price correlates negatively with view→cart (corr ≈ –0.56). Higher-priced items convert less.  
- **Action**:  
  - Introduce **financing options** or “buy now, pay later” for premium products.  
  - Offer **free shipping thresholds** (e.g. “Free shipping over \$50”) to offset sticker shock.

## Final Recommendations

1. **Streamline Checkout Flow**  
   - Simplify form fields, add guest-checkout, and pre-fill known user data.  
   - A/B test a one-page vs. multi-page checkout to boost the 4% cart→purchase rate.

2. **Time-Based Engagement Nudges**  
   - Trigger on-site reminders at ~7 min and ~17 min into a session.  
   - Send email/push notifications ~15–20 min after cart add for users who haven’t checked out.

3. **Prioritize Top Categories & Brands**  
   - Allocate more ad budget and premium placement to `apparel.glove` (33% conv.) and `stationery.cartrige` (20%).  
   - Partner with high-conv. brands like **eunyul**, **supertan**, **elskin** (40–44%) for featured bundles and promotions.

4. **Optimize Mid-Funnel for Mid-Tier Categories**  
   - For categories with ~30% view→cart but <10% cart→purchase (e.g., air_conditioner, bathroom.bath), A/B test CTAs and enrich pages with urgency badges and reviews.


5. **Leverage Pricing Strategies**  
   - Introduce financing or “buy now, pay later” on higher-priced items (negative price→cart correlation: –0.56).  
   - Offer free shipping thresholds (e.g., “Free shipping over \$50”) to reduce sticker shock.
   
6. **Capture & Act on Remove-from-Cart Data**  
   - Instrument `remove_from_cart` events to understand why users drop items.  
   - Deploy micro-surveys (“Why did you remove this?”) and rectify UX or pricing issues.

7. **Address Monthly Abandonment Spikes**  
   - October and January see the highest cart abandons—run targeted incentives (e.g., “10% off re-cart”) during these months.  
   - Use exit-intent pop-ups to recover cart abandons before they leave.

8. **Enable Session Persistence**  
   - Implement “Save for Later” and persistent carts so returning users can pick up where they left off.  
   - Follow up with reminder emails at 30 min and 12 hr intervals for in-cart users.

9. **Move High Drop-Off Products**  
    - Identify the top 20 SKUs with ~100% add-to-cart but no purchase; audit their pages for pricing, descriptions, and stock issues.  
    - Run flash sales or bundle discounts to clear these items.

---

By executing these data-driven actions—targeting the biggest funnel leaks, doubling down on what works, and filling measurement gaps—you’ll drive significant improvements in both conversion rates and overall revenue.  
